In [1]:
# scraping the data from the web
import pandas as pd
import numpy as np

from urllib.request import urlopen

from bs4 import BeautifulSoup
page_list=[
        "http://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=117;team=4343;type=trophy",
        "http://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=117;team=4347;type=trophy",
        "http://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=117;team=4344;type=trophy",
        "http://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=117;team=5845;type=trophy",
        "http://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=117;team=4342;type=trophy",
        "http://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=117;team=4788;type=trophy",
        "http://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=117;team=4341;type=trophy",
        "http://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=117;team=4346;type=trophy",
        "http://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=117;team=4787;type=trophy",
        "http://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=117;team=4345;type=trophy",
        "http://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=117;team=5843;type=trophy",
        "http://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=117;team=4340;type=trophy",
        "http://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=117;team=5143;type=trophy"
            ]
names = [       
    "Chennai Super Kings","Deccan Chargers","Delhi Daredevils","Gujarat Lions","Kings XI Punjab",
    "Kochi Tuskers Kerala","Kolkata Knight Riders","Mumbai Indians","Pune Warriors","Rajasthan Royals",
    "Rising Pune Supergiants","Royal Challengers Bangalore","Sunrisers Hyderabad"
    ]
ext = ".csv"
k=0
for i in page_list:
    page = urlopen(i)
    soup = BeautifulSoup(page)
    right_table=soup.find('table', class_='engineTable')
    l = []
    th=right_table.find_all("th")
    head=[hd.text for hd in th]
    for tr in right_table.findAll("tr"):
        td = tr.find_all('td')
        row = [tr.text for tr in td]
        l.append(row)
    df=pd.DataFrame(l)
    df.columns=head
    df.to_csv(names[k]+ext,index=False)
    k+=1
    
#after manual cleaning, concatenating the files 
# from glob import glob
# filenames=glob("*.csv")
# headers=["player_name","matches_played","innings_bowled","overs_bowled","maidens_earned","runs_conceded",
#          "wickets_taken","best_innings_bowling","bowling_average","economy_rate","bowling_strike_rate",
#          "four_wkts_in_an_inns","five_wkts_in_an_inns","catches_taken","stumpings_made"]   

# dfs=[pd.read_csv(x,names=headers) for x in filenames]
# combined=pd.concat(dfs)
# combined.to_csv("all_teams_bowling.csv",index=False)


/usr/lib/python3/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [2]:
from glob import glob
filenames=glob("*.csv")
dfs=[]
headers=["player_name","matches_played","innings_bowled","overs_bowled","maidens_earned","runs_conceded",
         "wickets_taken","best_innings_bowling","bowling_average","economy_rate","bowling_strike_rate",
         "four_wkts_in_an_inns","five_wkts_in_an_inns","catches_taken","stumpings_made"]   
for i in filenames:
    if(i[:-4] in names):
        df1=pd.read_csv(i)
        df1=df1.iloc[1:]
        if 'Span' in df1.columns:
            df1.drop(columns=['Span'],inplace=True)
        df1.columns=headers    
        print(df1.shape)
        dfs.append(df1)

combined=pd.concat(dfs)
combined.to_csv("all_teams_bowling.csv",index=False)
combined.shape

(106, 15)
(107, 15)
(62, 15)
(29, 15)
(46, 15)
(61, 15)
(120, 15)
(34, 15)
(99, 15)
(97, 15)
(100, 15)
(20, 15)
(72, 15)


(953, 15)

In [3]:
df = pd.read_csv("all_teams_bowling.csv")
df.shape

(953, 15)

In [4]:
i = df[df.runs_conceded=='-'].index #find the bowlers who haven't conceded any runs 

In [5]:
df.drop(i,inplace=True) #remove the bowlers who haven't conceded any runs 

In [6]:
#change the data types to actual data types
#eg:  runs_conceded was object(string) but it should be int
df['runs_conceded']=df['runs_conceded'].astype('int64')
df['wickets_taken']=df['wickets_taken'].astype('int64')
df['overs_bowled']=df['overs_bowled'].astype('float64')

In [7]:
#group the multiple data points of a player and sum the attributes
res = df.groupby('player_name').agg({'runs_conceded':'sum','wickets_taken':'sum','overs_bowled':'sum'}).reset_index()

In [8]:
#stats cal
res['bowling_average']=(res['runs_conceded']/res['wickets_taken']).round(2)
res['economy_rate']=(res['runs_conceded']/res['overs_bowled']).round(2)
res['bowling_strike_rate']=((res['overs_bowled']*6)/res['wickets_taken']).round(2)

In [9]:
res

,player_name,runs_conceded,wickets_taken,overs_bowled,bowling_average,economy_rate,bowling_strike_rate
0,A Ashish Reddy,396,18,43.4,22.000000,9.12,14.470000
1,A Chandila,242,11,39.0,22.000000,6.21,21.270000
2,A Choudhary,144,5,16.5,28.800000,8.73,19.800000
3,A Dananjaya,47,0,4.0,inf,11.75,inf
4,A Flintoff,105,2,11.0,52.500000,9.55,33.000000
5,A Kumble,1058,45,160.5,23.510000,6.59,21.400000
6,A Mishra,3525,146,476.5,24.140000,7.40,19.580000
7,A Mithun,472,7,48.0,67.430000,9.83,41.140000
8,A Nehra,2495,106,317.6,23.540000,7.86,17.980000
9,A Nel,31,1,3.0,31.000000,10.33,18.000000


In [10]:
res['bowling_average']=res['bowling_average'].replace(np.inf,np.nan)
res['bowling_strike_rate']=res['bowling_strike_rate'].replace(np.inf,np.nan)

In [11]:
res

,player_name,runs_conceded,wickets_taken,overs_bowled,bowling_average,economy_rate,bowling_strike_rate
0,A Ashish Reddy,396,18,43.4,22.00,9.12,14.47
1,A Chandila,242,11,39.0,22.00,6.21,21.27
2,A Choudhary,144,5,16.5,28.80,8.73,19.80
3,A Dananjaya,47,0,4.0,NaN,11.75,NaN
4,A Flintoff,105,2,11.0,52.50,9.55,33.00
5,A Kumble,1058,45,160.5,23.51,6.59,21.40
6,A Mishra,3525,146,476.5,24.14,7.40,19.58
7,A Mithun,472,7,48.0,67.43,9.83,41.14
8,A Nehra,2495,106,317.6,23.54,7.86,17.98
9,A Nel,31,1,3.0,31.00,10.33,18.00


In [12]:
res.to_csv('stats_bowling_1.csv',index=None)

In [13]:
res.shape

(377, 7)